In [1]:
import helper
import joblib
from sklearn.metrics import precision_score, f1_score

In [6]:

def aggregate_predict(models, X_test, y_test):
    metrics = []
    
    # Test the local models and show the metrics
    models_predicts = []
    for i in range(len(models)):
        local_model = joblib.load(models[i])
        predict = local_model.predict(X_test)
        models_predicts.append(predict)
        print(f"Local model {i + 1} prediction results:")
        precision = precision_score(y_test, predict)
        f1 = f1_score(y_test, predict)
        metrics.append([precision, f1])
        helper.display_metrics(y_test, predict)
    
    # Compute weighted metrics
    precision_weight = 0.6
    f1_weight = 0.4
    metrics_scores = []
    for precision, f1 in metrics:
        score = (precision * precision_weight) + (f1 * f1_weight)
        metrics_scores.append(score)
    
    total_score = sum(metrics_scores)
    weights = []
    for score in metrics_scores:
        weight = score / total_score
        weights.append(weight)
    
    # Compute weighted prediction
    weighted_predictions = []
    for predicts in zip(*models_predicts):
        weighted_sum = 0
        for pred, weight in zip(predicts, weights):
            weighted_sum += pred * weight
        if weighted_sum >= 0.3:
            weighted_prediction = 1
        else:
            weighted_prediction = 0
        weighted_predictions.append(weighted_prediction)

    print("Aggregated prediction results:")
    helper.display_metrics(y_test, weighted_predictions)

In [7]:
_, X_test, y_test = helper.load_testset(test_size=0.3)
models = ['./received_models/client_1.joblib', './received_models/client_2.joblib', './received_models/client_3.joblib']

aggregate_predict(
    models=models,
    X_test=X_test, 
    y_test=y_test,
)

Local model 1 prediction results:
---------------------
Accuracy : 0.95098245
Precision: 0.95069359
Recall   : 0.95098245
F1 Score : 0.94766767
---------------------

Local model 2 prediction results:
---------------------
Accuracy : 0.95198720
Precision: 0.95168883
Recall   : 0.95198720
F1 Score : 0.94884143
---------------------

Local model 3 prediction results:
---------------------
Accuracy : 0.95119263
Precision: 0.95098899
Recall   : 0.95119263
F1 Score : 0.94786276
---------------------

Aggregated prediction results:
---------------------
Accuracy : 0.97417377
Precision: 0.97414629
Recall   : 0.97417377
F1 Score : 0.97415978
---------------------


In [19]:
# def aggregate_predict(models, weights, X_test, y_test, threshold=0.3):
#     if len(models) != len(weights):
#         print("Error: Missing models or weights.")
#         return 
#     
#     # Test the local models and show the metrics
#     predicts = []
#     for i in range(len(models)):
#         local_model = joblib.load(models[i])
#         predict = local_model.predict(X_test)
#         predicts.append(predict)
#         print(f"Local model {i + 1} prediction results:")
#         helper.display_metrics(y_test, predict)
#     
#     # Give new predictions
#     new_predict = []
#     for i in range(len(y_test)):
#         weighted_sum = 0
#         
#         for weight, predict in zip(weights, predicts):
#             weighted_sum += weight * predict[i]
#     
#         combined_weight = sum(weights)
#         weighted_avg = weighted_sum / combined_weight
#             
#         if weighted_avg < threshold:
#             new_predict.append(0)
#         else:
#             new_predict.append(1)
# 
#     print("Aggregated prediction results:")
#     helper.display_metrics(y_test, new_predict)

In [20]:
filename = './received_models/ratio.txt'
ratio_dict = {}
with open(filename, 'r') as file:
    for line in file:
        key, value = line.strip().split(':')
        ratio_dict[int(key)] = float(value)

weights = [ratio_dict[1], ratio_dict[2], ratio_dict[3]]
weights

[0.86087805, 0.86141851, 0.86014645]

In [21]:
_, X_test, y_test = helper.load_testset(test_size=0.3)
models = ['./received_models/client_1.joblib', './received_models/client_2.joblib', './received_models/client_3.joblib']

aggregate_predict(
    models=models,
    X_test=X_test, 
    y_test=y_test,
)

Local model 1 prediction results:
---------------------
Accuracy : 0.95175652
Precision: 0.95148055
Recall   : 0.95175652
F1 Score : 0.94857914
---------------------

Local model 2 prediction results:
---------------------
Accuracy : 0.95080303
Precision: 0.95053143
Recall   : 0.95080303
F1 Score : 0.94746805
---------------------

Local model 3 prediction results:
---------------------
Accuracy : 0.95153097
Precision: 0.95125416
Recall   : 0.95153097
F1 Score : 0.94831781
---------------------

Aggregated prediction results:
---------------------
Accuracy : 0.97479405
Precision: 0.97477375
Recall   : 0.97479405
F1 Score : 0.97478376
---------------------
